In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import sys

# Configurações de visualização
plt.style.use('seaborn')
sns.set_palette("husl")
%matplotlib inline

# Configurar o notebook para exibir mais colunas
pd.set_option('display.max_columns', None)

# Adiciona o diretório raiz do projeto ao PYTHONPATH
project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Importações do projeto
from src.utils.technical_indicators import add_technical_indicators
from src.utils.config_loader import load_config
from src.utils.data_validator import DataValidator

# Carregar configurações
env_cfg = load_config('config/env_config.yaml')

# Definir diretório de dados
DATA_DIR = Path('data/raw')


# Análise Exploratória de Dados para Trading com RL

Este notebook contém a análise exploratória dos dados utilizados no projeto de trading com Reinforcement Learning. Vamos analisar os dados históricos, indicadores técnicos e padrões de mercado.

In [ ]:
def load_market_data(symbol='AVAX', quote='USDT', timeframe='1h'):
    """Carrega dados do mercado a partir do arquivo CSV"""
    filename = f"{symbol}{quote}_{timeframe}.csv"
    filepath = DATA_DIR / filename
    
    if not filepath.exists():
        raise FileNotFoundError(f"Arquivo {filename} não encontrado em {DATA_DIR}")
    
    df = pd.read_csv(filepath, parse_dates=['timestamp'])
    df.set_index('timestamp', inplace=True)
    
    # Validação dos dados usando DataValidator
    validator = DataValidator()
    validator.validate_dataframe(df, required_columns=['open', 'high', 'low', 'close', 'volume'])
    
    # Adiciona indicadores técnicos
    df = add_technical_indicators(df)
    return df

# Carregar dados de exemplo
try:
    df = load_market_data()
    print("Dados carregados com sucesso!")
    print(f"Período: {df.index.min()} até {df.index.max()}")
    print(f"Total de registros: {len(df)}")
    print("\nColunas disponíveis:")
    print(df.columns.tolist())
except Exception as e:
    print(f"Erro ao carregar dados: {e}")
    raise


## 1. Análise Estatística dos Preços

Vamos examinar as estatísticas descritivas dos preços e volume de negociação.

In [ ]:
# Estatísticas descritivas
print("Estatísticas descritivas dos preços:")
display(df[['open', 'high', 'low', 'close', 'volume']].describe())

# Visualização da série temporal de preços
plt.figure(figsize=(15, 7))
plt.plot(df.index, df['close'], label='Preço de Fechamento')
plt.title('Série Temporal de Preços')
plt.xlabel('Data')
plt.ylabel('Preço')
plt.legend()
plt.grid(True)
plt.show()


## 2. Análise de Indicadores Técnicos

Vamos analisar os principais indicadores técnicos utilizados na estratégia de trading.

In [ ]:
# Análise de Indicadores Técnicos
fig, axes = plt.subplots(3, 1, figsize=(15, 15))

# RSI
axes[0].plot(df.index, df['rsi_14'])
axes[0].axhline(y=70, color='r', linestyle='--')
axes[0].axhline(y=30, color='g', linestyle='--')
axes[0].set_title('RSI (14)')
axes[0].grid(True)

# MACD
axes[1].plot(df.index, df['macd'], label='MACD')
axes[1].plot(df.index, df['macd_signal'], label='Sinal')
axes[1].set_title('MACD')
axes[1].legend()
axes[1].grid(True)

# Médias Móveis
axes[2].plot(df.index, df['close'], label='Preço', alpha=0.5)
axes[2].plot(df.index, df['sma_20'], label='SMA 20')
axes[2].plot(df.index, df['ema_50'], label='EMA 50')
axes[2].set_title('Médias Móveis')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.show()


## 3. Análise de Volatilidade

Vamos examinar a volatilidade do mercado usando o indicador ATR (Average True Range).

In [ ]:
# Análise de Volatilidade
plt.figure(figsize=(15, 7))
plt.plot(df.index, df['atr_14'], label='ATR (14)')
plt.title('Average True Range - Medida de Volatilidade')
plt.xlabel('Data')
plt.ylabel('ATR')
plt.legend()
plt.grid(True)
plt.show()


## 4. Correlação entre Indicadores

Vamos analisar a correlação entre os diferentes indicadores técnicos.

In [ ]:
# Correlação entre indicadores
corr_matrix = df[['close', 'rsi_14', 'macd', 'atr_14', 'sma_20', 'ema_50']].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlação entre Indicadores')
plt.show()
